In [ ]:
import tkinter as tk
from tkinter import filedialog
import os
import subprocess

def ADD_TITLE(frame,title):
    frame.TITLE = tk.Label(frame,text=title)
    frame.TITLE.pack(side="top")

def ADD_SELF_DESTRUCT(frame):
    frame.SELF_DESTRUCT = tk.Button(frame,text="REMOVE ANALYSIS",fg="red",command=frame.destroy)
    frame.SELF_DESTRUCT.pack(side="right")
    
def ADD_DATASET_NAME(frame,default_text=""):
    frame.DATASET_FRAME = tk.Frame(frame)
    frame.DATASET_FRAME.pack(side="top",fill="both")
    frame.dataset_label = tk.Label(frame.DATASET_FRAME,text="Dataset Name")
    frame.dataset_label.pack(side="left")
    frame.dataset_entry = tk.Entry(frame.DATASET_FRAME)
    frame.dataset_entry.insert(0, default_text)
    frame.dataset_entry.pack(side="right")

def FILE_OUTPUT_FRAME(frame,default_text=""):
    frame.FILE_OUTPUT=tk.Frame(frame)
    frame.FILE_OUTPUT.pack(side="top",fill='both')
    frame.file_label = tk.Label(frame.FILE_OUTPUT,text="Output filename")
    frame.file_label.pack(side="left")
    frame.file_entry = tk.Entry(frame.FILE_OUTPUT)
    frame.file_entry.insert(0, default_text)
    frame.file_entry.pack(side="right")

def SELECTION_MASK(frame,label,default_text=""):
    FRAME=tk.Frame(frame)
    FRAME.pack(side="top",fill='both')
    rmsd_label = tk.Label(FRAME,text=label)
    rmsd_label.pack(side="left")
    selection_mask = tk.Entry(FRAME)
    selection_mask.insert(0, default_text)
    selection_mask.pack(side="right")
    return FRAME

class FileFrame(tk.Frame):
    def __init__(self,name,filetypes,master=None):
        super().__init__(master)
        self.filename = ""
        self.name     = name
        self.filetypes= filetypes
        self.pack(fill='both')
        self.button = tk.Button(self,text=self.name,command=self.GetFile)
        self.button.pack(side="left")
    def GetFile(self):
        self.filename = filedialog.askopenfilename(initialdir = os.getcwd(),
                                              title = "Select a File",
                                              filetypes = self.filetypes)
        self.button.configure(fg="green")
        tk.Label(self,text=self.filename).pack(side="right")
        
class RadioButtonBoolWithTextEntryFrame(tk.Frame):
    def __init__(self,name,cond_text,master=None):
        super().__init__(master)
        self.name     = name
        self.cond_text= cond_text
        self.entry = tk.Entry(self)
        self.custom_entry = ""
        self.last_val = ""
        self.custom_text = tk.Label(self,text=self.cond_text)
        self.values   = {"Yes":True,"No":False}
        self.boolean  = tk.BooleanVar(self,False)
        self.pack(fill='both')
        tk.Label(self, text=self.name).pack(side="left")
        for (text, value) in self.values.items():
            tk.Radiobutton(self, text = text, variable = self.boolean,
                value = value,command=self.update).pack(fill="x",side="right", ipady = 2,ipadx=2)
    def update(self):
        if self.boolean.get():
            self.entry.pack_forget()
            self.custom_text.pack_forget()
            self.entry.pack(side="right")
            self.custom_text.pack(side="right")
            self.entry.delete(10,tk.END)
            self.entry.insert(10,self.last_val)
        else:
            self.entry.pack_forget()
            self.custom_text.pack_forget()
            self.custom_entry = ""
        self.master.update_Text()
    def get(self):
        self.custom_entry = self.entry.get()
        self.last_val     = self.entry.get()
        return self.boolean.get()


In [ ]:
### ANALYSIS FRAMES
### RMSD vs. Time
class RMSD(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"RMSD vs. Time")
        ### RMSD Selection Mask Frame
        self.RMSD_FRAME = SELECTION_MASK(self,"RMSD Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self)
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        return f"rms first perres {self.selection_mask.get()} out {self.file_entry.get()}\n"

### Distance between two atoms vs. Time
class Distance(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Distance Analysis")
        ADD_DATASET_NAME(self,"d1")
        ### Atom Selections Frame
        self.ATOMS_FRAME = tk.Frame(self)
        self.ATOMS_FRAME.pack(side="top",fill='both')
        self.ATOM1_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 1 Selection Mask")
        self.ATOM2_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 2 Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,"Distances.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def command_string(self):
        dataset = self.dataset_entry.get()
        atom1 = self.ATOM1_FRAME.selection_mask.get()
        atom2 = self.ATOM2_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"distance {dataset} {atom1} {atom2} out {fileout}"

### Angle between three atoms vs. Time
class Angle(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Angle Analysis")
        ADD_DATASET_NAME(self,"a1")
        ### Atom Selections Frame
        self.ATOMS_FRAME = tk.Frame(self)
        self.ATOMS_FRAME.pack(side="top",fill='both')
        self.ATOM1_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 1 Selection Mask")
        self.ATOM2_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 2 Selection Mask")
        self.ATOM3_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 3 Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,"Angles.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def command_string(self):
        dataset = self.dataset_entry.get()
        atom1 = self.ATOM1_FRAME.selection_mask.get()
        atom2 = self.ATOM2_FRAME.selection_mask.get()
        atom3 = self.ATOM3_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"angle {dataset} {atom1} {atom2} {atom3} out {fileout}"
    
### Dihedral/Torsion between four atoms v. Time
class Torsion(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Dihedral/Torsion Analysis")
        ADD_DATASET_NAME(self,"t1")
        ### Atom Selections Frame
        self.ATOMS_FRAME = tk.Frame(self)
        self.ATOMS_FRAME.pack(side="top",fill='both')
        self.ATOM1_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 1 Selection Mask")
        self.ATOM2_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 2 Selection Mask")
        self.ATOM3_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 3 Selection Mask")
        self.ATOM4_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 4 Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,"Dihedrals.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def command_string(self):
        dataset = self.dataset_entry.get()
        atom1 = self.ATOM1_FRAME.selection_mask.get()
        atom2 = self.ATOM2_FRAME.selection_mask.get()
        atom3 = self.ATOM3_FRAME.selection_mask.get()
        atom4 = self.ATOM4_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"dihedral {dataset} {atom1} {atom2} {atom3} {atom4} out {fileout}"
    
### Nucleotide Sugar Puckering v. Time
class NucleotidePucker(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Nucleotide Sugar Pucker Analysis")
        ADD_DATASET_NAME(self,"pucker1")
        ### Atom Selections Frame
        self.ATOMS_FRAME = tk.Frame(self)
        self.ATOMS_FRAME.pack(side="top",fill='both')
        self.ATOM1_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Residue Number")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,"NucleotidePucker.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def command_string(self):
        dataset = self.dataset_entry.get()
        res_num = self.ATOM1_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"pucker {dataset} :{res_num}@C1’ :{res_num}@C2’ :{res_num}@C3’ :{res_num}@C4’ :{res_num}@O4’ range360 out {fileout}\n"

### RMSF By Residue
class RMSF(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"RMSF By Residue")
        ADD_DATASET_NAME(self,"RMSF_ByRes")
        ### RMSF Selection Mask Frame
        self.RMSF_FRAME = SELECTION_MASK(self,"RMSF Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self)
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        dataset = self.dataset_entry.get()
        sel_mask = self.RMSF_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"atomicfluct {dataset} {sel_mask} out {fileout} byres"
        

### Correlated Motion between Residues
class CorrelatedMotion(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Correlated Motion")
        ADD_DATASET_NAME(self,"corrpath")
        ### RMSF Selection Mask Frame
        self.CORREL_FRAME = SELECTION_MASK(self,"Correlation Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self)
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        dataset = self.dataset_entry.get()
        sel_mask = self.CORREL_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"matrix correl name {dataset} out {fileout} {sel_mask} byres"

### Normal Mode Analysis
class NormalModes(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Normal Modes Analysis")
        ADD_DATASET_NAME(self,"nma_data")
        ### RMSF Selection Mask Frame
        self.NMA_FRAME = SELECTION_MASK(self,"Normal Modes Selection Mask")
        self.NUM_MODES = SELECTION_MASK(self,"Number of Modes")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self)
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        dataset = self.dataset_entry.get()
        sel_mask = self.NMA_FRAME.selection_mask.get()
        num_modes = self.NUM_MODES.selection_mask.get()
        fileout = self.file_entry.get()
        com_str = f"matrix covar name {dataset} {sel_mask}\n"
        com_str+= f"diagmatrix {dataset} vecs {num_modes} reduce nmwiz mnwizvecs {num_modes} \\ \n"
        com_str+= f"nmwizfile {fileout} nmwizmask {sel_mask}\n"
        return com_str

### Hydrogen Bonding Analysis
class HydrogenBonds(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Hydrogen Bond Analysis")
        self.intramolecular = tk.BooleanVar(self,False)
        self.DISTANCE = SELECTION_MASK(self,"H-Bond cutoff distance",default_text="3.0")
        self.INTRAMOL = tk.Frame(self)
        self.label = tk.Label(self.INTRAMOL,text="Exclude intramolecular hydrogen bonds?")
        self.label.pack(side="left")
        for (text, value) in {"Yes":True,"No":False}.items():
            tk.Radiobutton(self.INTRAMOL, text = text, variable = self.intramolecular,
                value = value).pack(fill="x",side="right", ipady = 2,ipadx=2)
#         self.intramol = RadioButtonBoolWithTextEntryFrame(,"Selection Mask",master=self)
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,default="HBonds.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        nointramol = ""
        if self.intramolecular.get():
            nointramol = "nointramol"
        fileout = self.file_entry.get()
        distance = self.DISTANCE.selection_mask.get()
        return f"hbond dist {distance} avgout {fileout} {nointramol}"


### Radial Distribution Function
class RadialDistribution(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Radial Distribution Function")
        self.BINSPLIT = SELECTION_MASK(self,"Bin Separation",default_text="0.5")
        self.MAXDIST  = SELECTION_MASK(self,"Maximum Distance",default_text="10.0")
        self.SOLVENT  = SELECTION_MASK(self,"Solvent Molecule Mask",default_text=":WAT")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,default="HBonds.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        fileout = self.file_entry.get()
        split   = self.BINSPLIT.selection_mask.get()
        maxdist = self.MAXDIST.selection_mask.get()
        solvent = self.SOLVENT.selection_mask.get()
        return f"rdf out {fileout} {split} {maxdist} {solvent}"



In [ ]:
### Main Analysis Frame
class Analysis_Frame(tk.Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.master = master
        self.options=["SELECT ANALYSIS","RMSD","RMSF","Correlated Motion","Normal Modes",
                      "Hydrogen Bond Analysis","Radial Distribution Function",
                      "Distance","Angle","Dihedral/Torsion","Clustering"]
        self.analysis_type = tk.StringVar()
        self.analysis_type.set("SELECT ANALYSIS")
        self.pack(fill="both")
        
        ### ADD NEW ANALYSIS FRAME
        self.ADD_ANALYSIS_FRAME = tk.Frame(self)
        self.ADD_ANALYSIS_FRAME.pack(side="bottom")
        self.drop = tk.OptionMenu(self.ADD_ANALYSIS_FRAME ,self.analysis_type, *self.options)
        self.drop.pack(side="left")
        self.button = tk.Button(self.ADD_ANALYSIS_FRAME,text="ADD ANALYSIS",command=self.add_analysis)
        self.button.pack(side="left")

    def add_analysis(self):
        antype = self.analysis_type.get()
        self.ADD_ANALYSIS_FRAME.pack_forget()
        if antype == "RMSD":
            newframe = RMSD(self)
        if antype == "RMSF":
            newframe = RMSF(self)
        if antype == "Correlated Motion":
            newframe = CorrelatedMotion(self)
        if antype == "Normal Modes":
            newframe = NormalModes(self)
        if antype == "Hydrogen Bond Analysis":
            newframe = HydrogenBonds(self)
        if antype == "Radial Distribution Function":
            newframe = RadialDistribution(self)
        if antype == "Distance":
            newframe = Distance(self)
        if antype == "Angle":
            newframe = Angle(self)
        if antype == "Dihedral/Torsion":
            newframe = Torsion(self)
        if antype == "Clustering":
            newframe == Clustering(self)
        if antype != "SELECT ANALYSIS":
            newframe.pack(side="top")
        self.ADD_ANALYSIS_FRAME.pack(side="bottom")
        
    def __str__(self):
        return_string = ""
        for frame in self.pack_slaves():
            if getattr(frame,"command_string",False):
                return_string += frame.command_string()
        return return_string
        

In [89]:
class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        ### INTERNAL VARIABLES ###
        self.BooleanOptions = ["Autoimage","StripWater","StripIons"]
        self.master = master
        self.trajectory_file = ""
        self.prmtop_file     = ""
        self.autoimage_bool  = tk.BooleanVar(self,False)
        self.stripwater_bool = tk.BooleanVar(self,False)
        self.stripions_bool  = tk.BooleanVar(self,False)
        self.rmsf_sel_mask   = ""
        self.rmsd_sel_mask   = ""
        self.cations         = "K+"
        self.anions          = "Cl-"

        ### FRAMES ###
        self.frame_run_or_quit = tk.Frame(self)
        self.display_commands_frame = tk.Frame(self)
        self.analyses_frame = tk.Frame(self)
        
        ### FINAL INITIALIZATION OF PROGRAM ###
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        ### FileFrames
        self.prmtopfileframe = FileFrame("Prmtop File",(("AMBER Parm7","*.prmtop*"),("all files","*.*")),master=self)
        self.trajectoryfileframe  = FileFrame("Trajectory File",(("CHARMM DCD","*.dcd*"),("AMBER MDCRD","*.mdcrd*"),("all files","*.*")),master=self)

        ### RadioButton Options
        self.frame_autoimage = RadioButtonBoolWithTextEntryFrame("Autoimage?","Selection Mask",master=self)
        self.frame_stripwater = RadioButtonBoolWithTextEntryFrame("Strip solvent?","Solvent Mask",master=self)
        self.frame_stripions = RadioButtonBoolWithTextEntryFrame("Strip counterions?","Counterions Mask",master=self)

        # Add Analyses
#         self.analyses_frame.pack(fill="both")
        self.ANALYSES = Analysis_Frame(self)
        self.ANALYSES.pack(fill="both")

        # DISPLAY CURRENT COMMANDS
        self.display_commands_frame.pack(fill="both")
        self.T = tk.Text(self.display_commands_frame, height=10, width=100)
        self.commands_list="this is a test of commands\nsecondline\n now a third line\n  four score and seven whatever"
        self.T.pack()
        
        # RUN CPPTRAJ or QUIT Frame
        self.frame_run_or_quit.pack(fill="both")
        self.quit = tk.Button(self.frame_run_or_quit,text="QUIT",fg="red",command=self.master.destroy)
        self.quit.pack(side="right")
        self.run_cpptraj = tk.Button(self.frame_run_or_quit,text="RUN CPPTRAJ",fg="green",command=self.run_cpptraj)
        self.run_cpptraj.pack(side="right")
        self.update_text = tk.Button(self.frame_run_or_quit,text="UPDATE_TEXT",fg="blue",command=self.update_Text)
        self.update_text.pack(side="right")
        
        
    def run_cpptraj(self):
        self.update_Text()
        with open("cpptraj.in","w") as f:
            f.write(self.commands_list)
        f.close()
        subprocess.call("cpptraj < cpptraj.in > cpptraj.out", shell=True)
        # generate cpptraj.in based on all variables, then call cpptraj.
        pass
    
    def update_Text(self):
        self.commands_list  = f"parm {self.prmtopfileframe.filename}\n"
        self.commands_list += f"trajin {self.trajectoryfileframe.filename}\n"
        if self.frame_autoimage.get():
            self.commands_list+=f"autoimage anchor {self.frame_autoimage.custom_entry}\n"
        if self.frame_stripwater.get():
            self.commands_list+=f"strip {self.frame_stripwater.custom_entry}\n"
        if self.frame_stripions.get():
            self.commands_list+=f"strip {self.frame_stripions.custom_entry}\n"
        ### Add Analyses
#         print(self.ANALYSES.pack_slaves())
        for frame in self.ANALYSES.pack_slaves():
            self.commands_list +=str(frame)
        self.commands_list += "run\n"
        
        ### UPDATE DISPLAYED TEXT ###    
        self.T.delete(1.0,tk.END)
        self.T.insert(tk.END,self.commands_list) 
        self.T.config(height=len(self.commands_list.split("\n"))+1)
        self.T.pack()


### RUN MAIN APPLICATION
root = tk.Tk()
root.title("CPPTRAJ Analysis Generator")
app = Application(master=root)
app.mainloop()





In [ ]:






def ClusteringKmeans(num_clusters=10):
    return f"""
### Clustering (Requires datasets to cluster on beforehand)
cluster kmeans clusters {num_clusters} \ # specifies 5 clusters desired.
data t1,a1 \  # This is where the NAMED datasets to be clustered on are listed (comma-separated)
out cnumvtime.dat \ # specifies which cluster number each frame belongs to.
summary clustsummary.txt \ # summarizes each cluster
info clustinfo.txt \ # writes additional cluster information
repout clust \ # prefix of output cluster coordinates (clust.00001.rst7)
repfmt restart \ # specifies the ".rst7" format and file extension
bestrep centroid \ # representative frame for each cluster is closest to centroid of cluster.
savenreps 1 \ # save one frame per cluster
clustersvtime clustvtime.txt \ # writes the number of unique clusters seen over time below.
cvtwindow 50 \ # timewindow is 50 frames for clustersvtime above.
cpopvtime cpopvtime.dat # All cluster populations vs. time.
"""

